# Binary Decision Diagrams (BDDs)

## 1. Introduction 

### What are Binary Decision Diagrams?

A **Binary Decision Diagram (BDD)** is a data structure for representing Boolean functions efficiently. 

**Key Idea:** Instead of storing a truth table with 2^n rows for n variables, we store a directed acyclic graph that can be exponentially smaller.



### Why Do We Need BDDs?

**Problem:** Checking if two propositional formulas are equivalent requires:
- Constructing truth tables (2^n rows)
- Comparing every row
- **Time complexity:** O(2^n)
- **Space complexity:** O(2^n)

**Solution:** BDDs provide:
- Canonical representation (reduced BDDs are unique) Canonical forms enable O(1) equivalence checking
- Efficient equivalence checking (graph isomorphism)
- Compact storage (often much smaller than 2^n)
- Fast logical operations



### Real-World Applications

- **Hardware Verification:** Checking if circuit designs are equivalent
- **Model Checking:** Verifying software properties
- **SAT Solving:** Determining formula satisfiability
- **Formal Methods:** Proving program correctness

In [35]:
import os
import sys
import time


# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))


from bdd.node import BDDNode, TERMINAL_TRUE, TERMINAL_FALSE
from bdd.truth_table import Interpretation, TruthTable, PartialInterpretation
from bdd.formula import (
    Formula, Variable, Not, And, Or, Implies, Iff,
    FormulaParser
)
from bdd.diagram import BDD, create_bdd_from_string
from bdd.reduction import BDDReducer, reduce_bdd
from bdd.operations import (
    BDDOperations,
    bdd_and, bdd_or, bdd_not, bdd_xor, bdd_implies, bdd_iff,
    are_equivalent
)
from bdd.visualizer import visualize_bdd, compare_bdds, BDDVisualizer



## 2. Propositional Logic Semantics



### 2.1 Syntax and Semantics

**Propositional Logic** consists of:
- **Atoms (Variables):** p, q, r, ...
- **Logical Connectives:**
  - ¬ (NOT)
  - ∧ (AND)
  - ∨ (OR)
  - → (IMPLIES)
  - ↔ (IFF)


### 2.2 Interpretations

An **interpretation** I assigns truth values to propositional variables:

$$I: \{p, q, r, ...\} \rightarrow \{T, F\}$$


### 2.3 Truth Tables

A **truth table** enumerates all possible interpretations and their results.

For n variables, there are **2^n interpretations**.

**Example: p ∨ q**

| p | q | p ∨ q |
|---|---|-------|
| F | F | F     |
| F | T | T     |
| T | F | T     |
| T | T | T     |



### 2.4 Semantic Properties

- **Satisfiable:** ∃ interpretation I such that I(A) = T
- **Valid (Tautology):** ∀ interpretations I, I(A) = T
- **Unsatisfiable (Contradiction):** ∀ interpretations I, I(A) = F
- **Equivalent:** A ≡ B iff ∀I, I(A) = I(B)

In [36]:
#  Demonstrate Truth Tables
print("Example: Truth Table for p ∨ (q ∧ r)")

# Create formula
formula = FormulaParser.parse_formula("p | (q & r)")
print(f"Formula: {formula}")
print(f"Variables: {formula.get_variables()}")

# Generate truth table
def eval_func(interp):
    return formula.evaluate(interp)

table = TruthTable.from_function(['p', 'q', 'r'], eval_func)
print("\nTruth Table:")
print(table)

print(f"\nSatisfiable: {table.is_satisfiable()}")
print(f"Valid: {table.is_tautology()}")
print(f"Number of models: {len(table.get_models())}")

Example: Truth Table for p ∨ (q ∧ r)
Formula: (p ∨ (q ∧ r))
Variables: {'r', 'q', 'p'}

Truth Table:
p | q | r | Result
------------------
F | F | F |   F
F | F | T |   F
F | T | F |   F
F | T | T |   T
T | F | F |   T
T | F | T |   T
T | T | F |   T
T | T | T |   T

Satisfiable: True
Valid: False
Number of models: 5


## 3. The Complexity Problem



### 3.1 The Exponential Blow-Up

**Problem:** Truth tables grow exponentially with the number of variables.

| Variables (n) | Interpretations (2^n) | Memory Required |
|--------------|----------------------|-----------------|
| 10           | 1,024                | ~1 KB           |
| 20           | 1,048,576            | ~1 MB           |
| 30           | 1,073,741,824        | ~1 GB           |
| 40           | 1,099,511,627,776    | ~1 TB           |

**Conclusion:** Truth tables are **infeasible for large formulas**.



### 3.2 Decision Problems

**Satisfiability (SAT):**
- Input: Formula A
- Output: Is A satisfiable?
- **Complexity:** NP-complete

**Validity:**
- Input: Formula A
- Output: Is A valid?
- **Complexity:** co-NP-complete

**Equivalence:**
- Input: Formulas A, B
- Output: Is A ≡ B?
- **Complexity:** co-NP-complete



### 3.3 Why BDDs Help

BDDs provide:
1. **Canonical representation** → Equivalence in O(1) after reduction
2. **Compact encoding** → Often polynomial size instead of exponential
3. **Efficient operations** → Apply algorithm with memoization

**Key Insight:** BDDs trade worst-case exponential space for average-case polynomial space.

In [37]:

print("Demonstrating Exponential Growth")

results = []

for n in range(2, 8):
    # Create formula with n variables
    vars_list = [f"x{i}" for i in range(n)]
    formula_str = " | ".join([f"({vars_list[i]} & {vars_list[i+1]})" 
                              for i in range(0, n-1, 2)])
    
    start = time.time()
    bdd = create_bdd_from_string(formula_str)
    nodes_before = bdd.count_nodes()
    bdd.reduce()
    nodes_after = bdd.count_nodes()
    elapsed = time.time() - start
    
    truth_table_size = 2 ** n
    
    print(f"n={n}: Truth table size={truth_table_size:>4}, "
          f"BDD nodes={nodes_after:>3}, "
          f"Reduction: {truth_table_size/nodes_after:.1f}x, "
          f"Time: {elapsed*1000:.2f}ms")
    
    results.append((n, truth_table_size, nodes_after))

print("\n✅ BDDs provide exponential compression!")

Demonstrating Exponential Growth
n=2: Truth table size=   4, BDD nodes=  4, Reduction: 1.0x, Time: 0.18ms
n=3: Truth table size=   8, BDD nodes=  4, Reduction: 2.0x, Time: 0.11ms
n=4: Truth table size=  16, BDD nodes=  6, Reduction: 2.7x, Time: 0.54ms
n=5: Truth table size=  32, BDD nodes=  6, Reduction: 5.3x, Time: 0.21ms
n=6: Truth table size=  64, BDD nodes=  8, Reduction: 8.0x, Time: 0.63ms
n=7: Truth table size= 128, BDD nodes=  8, Reduction: 16.0x, Time: 0.73ms

✅ BDDs provide exponential compression!


## 4. Binary Decision Trees

### 4.1 From Truth Tables to Trees

**Idea:** Reduce truth table redundancy by representing it as a decision tree.

Each node represents a variable decision:
- **Solid edge (→):** Variable = True
- **Dotted edge (⇢):** Variable = False
- **Leaves:** Terminal values (T or F)



### 4.2 Example: p ∨ (q ∧ r)

**Full Truth Table (8 rows):**
```
p | q | r | Result
--|---|---|-------
F | F | F | F
F | F | T | F
F | T | F | F
F | T | T | T
T | F | F | T
T | F | T | T
T | T | F | T
T | T | T | T
```

**As Binary Decision Tree:**
- Each branch represents one interpretation
- Depth = number of variables
- 2^n leaves (one per interpretation)



### 4.3 Variable Ordering

**Definition 5.1 (from textbook):** A BDD is constructed with a **fixed variable ordering**.

**Example orderings for {p, q, r}:**
- [p, q, r] - lexicographic
- [q, p, r] - alternative
- [r, q, p] - reverse

**Important:** Different orderings can produce different-sized BDDs!


In [38]:
# Build and Visualize Initial BDD
print("Building Initial BDD for p | (q & r)")
print("This is unreduced BDD ")

# Create BDD
formula = FormulaParser.parse_formula("p | (q & r)")
bdd_unreduced = BDD(formula, variable_order=['p', 'q', 'r'])

print(f"Formula: {formula}")
print(f"Variable order: {bdd_unreduced.variable_order}")
print(f"Number of nodes: {bdd_unreduced.count_nodes()}")

# Test evaluation
test_cases = [
    ({'p': False, 'q': False, 'r': False}, False),
    ({'p': False, 'q': True, 'r': True}, True),
    ({'p': True, 'q': False, 'r': False}, True),
]

print("\nTesting some interpretations:")
for assignments, expected in test_cases:
    interp = Interpretation(assignments)
    result = bdd_unreduced.evaluate(interp)
    status = "Pass" if result == expected else "Failed"
    print(f"  {status} {assignments} → {result} (expected {expected})")

# Visualize
print("\n Creating visualization: bdd_unreduced.png")
visualize_bdd(bdd_unreduced, filename='bdd_unreduced', view=False)
print(" Unreduced BDD created with 9 nodes")

Building Initial BDD for p | (q & r)
This is unreduced BDD 
Formula: (p ∨ (q ∧ r))
Variable order: ['p', 'q', 'r']
Number of nodes: 9

Testing some interpretations:
  Pass {'p': False, 'q': False, 'r': False} → False (expected False)
  Pass {'p': False, 'q': True, 'r': True} → True (expected True)
  Pass {'p': True, 'q': False, 'r': False} → True (expected True)

 Creating visualization: bdd_unreduced.png
 Unreduced BDD created with 9 nodes


## 5. Shannon Expansion

### 5.1 The Shannon Decomposition

**Theorem (Shannon):** Any Boolean function f(x₁, ..., xₙ) can be decomposed as:

$$f = x_i \cdot f_{x_i=1} \vee \neg x_i \cdot f_{x_i=0}$$

Where:
- $f_{x_i=1}$ is f with xᵢ replaced by 1 (True)
- $f_{x_i=0}$ is f with xᵢ replaced by 0 (False)

**Notation:** 
- f[xᵢ ← 1] = cofactor when xᵢ = True
- f[xᵢ ← 0] = cofactor when xᵢ = False

### 5.2 Example: p ∨ (q ∧ r)

**Expand on variable p:**

$$f = p \cdot f_{p=1} \vee \neg p \cdot f_{p=0}$$

- f[p ← 1] = 1 ∨ (q ∧ r) = **1** (True)
- f[p ← 0] = 0 ∨ (q ∧ r) = **q ∧ r**

**Therefore:**
$$p \vee (q \wedge r) = p \cdot T \vee \neg p \cdot (q \wedge r)$$

### 5.3 Recursive Application

Shannon expansion is applied **recursively** to build the BDD:

1. Choose variable xᵢ (according to variable order)
2. Create node labeled xᵢ
3. Recursively compute left child: f[xᵢ ← 0]
4. Recursively compute right child: f[xᵢ ← 1]
5. Base case: reach constant True or False

### 5.4 Implementation in BDD Operations

Shannon expansion is used in the **Apply algorithm** for combining BDDs:

**Apply(op, f, g):**
- If f and g are terminals: return op(f, g)
- Choose lowest variable x in {f, g}
- Return: Node(x, Apply(op, f[x←0], g[x←0]), Apply(op, f[x←1], g[x←1]))

This allows us to compute f ∧ g, f ∨ g, etc. and other operations directly on BDDs without converting back to formulas!"

In [39]:

# Cell 10 (Code) - Demonstrate Shannon Expansion
print("Demonstrating Shannon Expansion")

# Create simple BDDs
bdd_p = create_bdd_from_string("p")
bdd_q = create_bdd_from_string("q")

print("Building: p ∨ q using Shannon expansion (Apply algorithm)")

# Apply OR operation which internally uses Apply with Shannon expansion
bdd_result = bdd_or(bdd_p, bdd_q)

print(f"Result nodes: {bdd_result.count_nodes()}")

# Verify correctness
test_cases = [
    ({'p': False, 'q': False}, False),
    ({'p': False, 'q': True}, True),
    ({'p': True, 'q': False}, True),
    ({'p': True, 'q': True}, True),
]

print("\nVerifying Shannon expansion result:")
all_correct = True
for assignments, expected in test_cases:
    interp = Interpretation(assignments)
    result = bdd_result.evaluate(interp)
    status = "✓" if result == expected else "✗"
    print(f"  {status} {assignments} → {result}")
    if result != expected:
        all_correct = False

print(f"\n{'✅' if all_correct else '❌'} Shannon expansion correctly implemented!")

Demonstrating Shannon Expansion
Building: p ∨ q using Shannon expansion (Apply algorithm)
Result nodes: 4

Verifying Shannon expansion result:
  ✓ {'p': False, 'q': False} → False
  ✓ {'p': False, 'q': True} → True
  ✓ {'p': True, 'q': False} → True
  ✓ {'p': True, 'q': True} → True

✅ Shannon expansion correctly implemented!


## 6. BDD Reduction Algorithm

### 6.1 Algorithm 5.3 (from textbook, page 99)

**Goal:** Reduce BDD size by eliminating redundancy.

**Algorithm Reduce:**

**Input:** Binary decision diagram bdd  
**Output:** Reduced binary decision diagram bdd'

**Steps:**

1. **Merge duplicate leaves:** If multiple leaves have the same value, keep only one of each (one T, one F)

2. **Remove redundant nodes (Step 1):** If both outgoing edges of a node point to the same child, delete the node and redirect incoming edges to the child

3. **Merge isomorphic sub-BDDs (Step 2):** If two nodes are roots of identical sub-BDDs, delete one and redirect incoming edges to the other

**Termination:** The algorithm repeats until no more reductions are possible. The result is a canonical form - unique for any given formula and variable ordering

### 6.2 Example Reduction

**Before reduction:**
```
      p
     / \
    q   q
   / \ / \
  F  T T  T
```

**After Step 1 (merge leaves):**
```
      p
     / \
    q   q
   / \ / \
  F    T        →  (merge T terminals)
```

**After Step 2 (remove redundant right q):**
```
      p
     / \
    q   \
   / \   \
  F     T
```

### 6.3 Unique Table

To efficiently detect isomorphic sub-BDDs (Step 2), we use a **hash table**:

- **Key:** (variable_label, low_child_id, high_child_id)
- **Value:** BDD node

**Invariant:** No two nodes in the table have the same key

**Time Complexity:** O(n) for reduction with n nodes, thanks to the hash table providing O(1) lookups."


In [40]:
# Demonstrate Reduction
print("Demonstrating BDD Reduction (Algorithm 5.3)")

# Create BDD
formula = FormulaParser.parse_formula("p | (q & r)")
bdd = BDD(formula, variable_order=['p', 'q', 'r'])

print(f"Formula: {formula}")
print(f"Before reduction: {bdd.count_nodes()} nodes")

# Perform reduction
stats = bdd.reduce()

print(f"After reduction: {bdd.count_nodes()} nodes")
print(f"\nReduction statistics:")
print(f"  - Nodes removed (Step 1): {stats['nodes_removed']}")
print(f"  - Nodes merged (Step 2): {stats['nodes_merged']}")
print(f"  - Total reduced: {stats['total_reduced']}")
print(f"  - Reduction ratio: {stats['nodes_before']/stats['nodes_after']:.2f}x")

# Verify correctness (Theorem 5.5)
print("\n Verifying correctness (Theorem 5.5):")
print("Testing all 8 interpretations...")

formula_direct = FormulaParser.parse_formula("p | (q & r)")
from bdd.truth_table import TruthTable

table = TruthTable(['p', 'q', 'r'])
all_match = True

for interp in table.generate_all_interpretations():
    bdd_result = bdd.evaluate(interp)
    direct_result = formula_direct.evaluate(interp)
    
    if bdd_result != direct_result:
        print(f"  Mismatch at {interp}")
        all_match = False

if all_match:
    print("  All interpretations match! Theorem 5.5 verified.")
else:
    print("  Reduction introduced errors!")

# Visualize
print("\n Creating visualization: bdd_reduced.png")
visualize_bdd(bdd, filename='bdd_reduced', view=False)


Demonstrating BDD Reduction (Algorithm 5.3)
Formula: (p ∨ (q ∧ r))
Before reduction: 9 nodes
After reduction: 5 nodes

Reduction statistics:
  - Nodes removed (Step 1): 4
  - Nodes merged (Step 2): 0
  - Total reduced: 4
  - Reduction ratio: 1.80x

 Verifying correctness (Theorem 5.5):
Testing all 8 interpretations...
  All interpretations match! Theorem 5.5 verified.

 Creating visualization: bdd_reduced.png


'bdd_reduced.png'

## 7. Theorem 5.5: Correctness of Reduction

### 7.1 Statement

**Theorem 5.5 (from textbook):** The reduced BDD bdd' returned by algorithm Reduce is logically equivalent to the input BDD bdd.

**Formally:**
$$\forall I: v_I(bdd) = v_I(bdd')$$

Where vᵢ is the evaluation function under interpretation I.

**This is crucial**  - it guarantees that reduction preserves semantics. We make the BDD smaller without changing what it computes.

### 7.2 Proof Sketch

**Lemma 1:** Merging duplicate terminals preserves semantics  
*Proof:* Both point to the same truth value

**Lemma 2:** Removing redundant nodes preserves semantics  
*Proof:* If low = high, then node's value = child's value regardless of variable assignment

**Lemma 3:** Merging isomorphic sub-BDDs preserves semantics  
*Proof:* Isomorphic sub-BDDs compute the same function by definition

**Conclusion:** Each step preserves semantics, therefore the entire algorithm preserves semantics.

### 7.3 Canonicity

**The resulting outcome:** For a fixed variable ordering, the reduced BDD is **unique** (canonical).

**Consequence:** Two formulas are equivalent if and only if their reduced BDDs (with same variable order) are isomorphic.

**Application:** Equivalence checking reduces to graph isomorphism, just compare the reduced BDDs! This is incredibly efficient compared to truth table comparison.

### 7.4 Practical Verification

We verify Theorem 5.5 by:
1. Building BDD from formula
2. Reducing the BDD
3. Testing all 2^n interpretations
4. Comparing results with direct formula evaluation


In [41]:

# Comprehensive Theorem 5.5 Verification
print("Comprehensive Verification of Theorem 5.5")


test_formulas = [
    ("p & q", "Simple conjunction"),
    ("p | q", "Simple disjunction"),
    ("p | (q & r)", "Textbook example"),
    ("(p & q) | (p & r)", "Distributive law"),
    ("p | ~p", "Tautology"),
    ("p & ~p", "Contradiction"),
    ("(p -> q) <-> (~p | q)", "Logical equivalence"),
]

all_verified = True

for formula_str, description in test_formulas:
    print(f"\n{description}: {formula_str}")
    
    # Build and reduce BDD
    bdd = create_bdd_from_string(formula_str)
    nodes_before = bdd.count_nodes()
    bdd.reduce()
    nodes_after = bdd.count_nodes()
    
    # Parse formula for direct evaluation
    formula = FormulaParser.parse_formula(formula_str)
    variables = sorted(formula.get_variables())
    
    # Test all interpretations
    table = TruthTable(variables)
    mismatches = 0
    
    for interp in table.generate_all_interpretations():
        bdd_result = bdd.evaluate(interp)
        formula_result = formula.evaluate(interp)
        
        if bdd_result != formula_result:
            mismatches += 1
    
    status = "Pass" if mismatches == 0 else "Failed"
    print(f"  {status} {nodes_before} → {nodes_after} nodes, "
          f"{2**len(variables)} interpretations tested, {mismatches} errors")
    
    if mismatches > 0:
        all_verified = False

print("\n")
if all_verified:
    print(" Theorem 5.5 VERIFIED: All reductions preserve semantics!")
else:
    print(" Theorem 5.5 VIOLATED: Errors detected!")


Comprehensive Verification of Theorem 5.5

Simple conjunction: p & q
  Pass 5 → 4 nodes, 4 interpretations tested, 0 errors

Simple disjunction: p | q
  Pass 5 → 4 nodes, 4 interpretations tested, 0 errors

Textbook example: p | (q & r)
  Pass 9 → 5 nodes, 8 interpretations tested, 0 errors

Distributive law: (p & q) | (p & r)
  Pass 9 → 5 nodes, 8 interpretations tested, 0 errors

Tautology: p | ~p
  Pass 2 → 1 nodes, 2 interpretations tested, 0 errors

Contradiction: p & ~p
  Pass 2 → 1 nodes, 2 interpretations tested, 0 errors

Logical equivalence: (p -> q) <-> (~p | q)
  Pass 4 → 1 nodes, 4 interpretations tested, 0 errors


 Theorem 5.5 VERIFIED: All reductions preserve semantics!


## 8. BDD Operations

### 8.1 The Apply Algorithm

The **Apply** algorithm computes binary operations on BDDs:

**Apply(op, f, g):**
```python
if f and g are both terminals:
    return op(f.value, g.value)

if f is terminal:
    var = g.variable
    f_low = f_high = f
    g_low, g_high = g.low, g.high
elif g is terminal:
    var = f.variable
    f_low, f_high = f.low, f.high
    g_low = g_high = g
else:
    var = min(f.variable, g.variable)  # by ordering (this ensures consistency)
    if f.variable == var:
        f_low, f_high = f.low, f.high
    else:
        f_low = f_high = f
    if g.variable == var:
        g_low, g_high = g.low, g.high
    else:
        g_low = g_high = g

low_result = Apply(op, f_low, g_low)
high_result = Apply(op, f_high, g_high)

if low_result == high_result:
    return low_result  # Redundancy elimination

return MakeNode(var, low_result, high_result)
```

**Key Features:**
- Uses Shannon expansion recursively
- Memoization (caching) for efficiency prevents recomputing the same sub-problems
- Automatic reduction during construction


### 8.2 Supported Operations

| Operation | Symbol | Apply Call |
|-----------|--------|------------|
| AND       | ∧      | Apply(AND, f, g) |
| OR        | ∨      | Apply(OR, f, g) |
| NOT       | ¬      | Negate all terminals |
| XOR       | ⊕      | Apply(XOR, f, g) |
| IMPLIES   | →      | Apply(IMPLIES, f, g) |
| IFF       | ↔      | Apply(IFF, f, g) |


In [42]:
# Demonstrate BDD Operations
print("Demonstrating BDD Operations")

# Create base BDDs
bdd_p = create_bdd_from_string("p")
bdd_q = create_bdd_from_string("q")
bdd_r = create_bdd_from_string("r")

print("Base BDDs created: p, q, r\n")

# Test all operations
operations = [
    ("AND", bdd_and, bdd_p, bdd_q, "p ∧ q"),
    ("OR", bdd_or, bdd_p, bdd_q, "p ∨ q"),
    ("NOT", lambda b, _: bdd_not(b), bdd_p, None, "¬p"),
    ("XOR", bdd_xor, bdd_p, bdd_q, "p ⊕ q"),
    ("IMPLIES", bdd_implies, bdd_p, bdd_q, "p → q"),
]

for op_name, op_func, bdd1, bdd2, description in operations:
    if bdd2 is None:
        result = op_func(bdd1, None)
    else:
        result = op_func(bdd1, bdd2)
    
    result.reduce()
    print(f"{op_name:8} | {description:10} → {result.count_nodes()} nodes")

# Complex operation
print("\n--- Complex Operation ---")
print("Computing: (p ∧ q) ∨ (p ∧ r)")

pq = bdd_and(bdd_p, bdd_q)
pr = bdd_and(bdd_p, bdd_r)
complex_result = bdd_or(pq, pr)
complex_result.reduce()

print(f"Result: {complex_result.count_nodes()} nodes")

# Verify equivalence with distributive law
print("\nVerifying: (p ∧ q) ∨ (p ∧ r) ≡ p ∧ (q ∨ r)")

qr = bdd_or(bdd_q, bdd_r)
distributive = bdd_and(bdd_p, qr)
distributive.reduce()

equivalent = are_equivalent(complex_result, distributive)
print(f"Equivalent: {equivalent} True" if equivalent else "Not equivalent ")

Demonstrating BDD Operations
Base BDDs created: p, q, r

AND      | p ∧ q      → 4 nodes
OR       | p ∨ q      → 4 nodes
NOT      | ¬p         → 3 nodes
XOR      | p ⊕ q      → 5 nodes
IMPLIES  | p → q      → 4 nodes

--- Complex Operation ---
Computing: (p ∧ q) ∨ (p ∧ r)
Result: 5 nodes

Verifying: (p ∧ q) ∨ (p ∧ r) ≡ p ∧ (q ∨ r)
Equivalent: True True



## 9. Implementation Overview

In [43]:

# Full Example: Building, Reducing, Operating
print("Complete BDD Workflow Example")
print("\n Task: Verify (p ∧ q) ∨ r ≡ (p ∨ r) ∧ (q ∨ r) (Distributive Law)\n")

# Step 1: Build first formula
print("Step 1: Build (p ∧ q) ∨ r")
formula1_str = "(p & q) | r"
bdd1 = create_bdd_from_string(formula1_str)
print(f"  Unreduced: {bdd1.count_nodes()} nodes")

stats1 = bdd1.reduce()
print(f"  Reduced: {bdd1.count_nodes()} nodes (removed {stats1['total_reduced']})")

# Step 2: Build second formula
print("\nStep 2: Build (p ∨ r) ∧ (q ∨ r)")
formula2_str = "(p | r) & (q | r)"
bdd2 = create_bdd_from_string(formula2_str)
print(f"  Unreduced: {bdd2.count_nodes()} nodes")

stats2 = bdd2.reduce()
print(f"  Reduced: {bdd2.count_nodes()} nodes (removed {stats2['total_reduced']})")

# Step 3: Check equivalence
print("\nStep 3: Check Equivalence")
equivalent = are_equivalent(bdd1, bdd2)
print(f"  Result: {equivalent}")

if equivalent:
    print("  Formulas are equivalent (Distributive Law verified)!")
else:
    print("   Formulas are NOT equivalent!")

# Step 4: Verify by truth table
print("\nStep 4: Verify by checking all interpretations")
formula1 = FormulaParser.parse_formula(formula1_str)
formula2 = FormulaParser.parse_formula(formula2_str)

table = TruthTable(['p', 'q', 'r'])
all_match = True

for interp in table.generate_all_interpretations():
    val1 = formula1.evaluate(interp)
    val2 = formula2.evaluate(interp)
    
    if val1 != val2:
        print(f"  Mismatch at {interp}: {val1} ≠ {val2}")
        all_match = False

if all_match:
    print(f"   All {2**3} interpretations match!")

# Step 5: Properties
print("\nStep 5: Check Properties")
print(f"  BDD1 satisfiable: {bdd1.is_satisfiable()}")
print(f"  BDD1 valid: {bdd1.is_valid()}")
print(f"  BDD2 satisfiable: {bdd2.is_satisfiable()}")
print(f"  BDD2 valid: {bdd2.is_valid()}")

Complete BDD Workflow Example

 Task: Verify (p ∧ q) ∨ r ≡ (p ∨ r) ∧ (q ∨ r) (Distributive Law)

Step 1: Build (p ∧ q) ∨ r
  Unreduced: 9 nodes
  Reduced: 5 nodes (removed 4)

Step 2: Build (p ∨ r) ∧ (q ∨ r)
  Unreduced: 9 nodes
  Reduced: 5 nodes (removed 4)

Step 3: Check Equivalence
  Result: True
  Formulas are equivalent (Distributive Law verified)!

Step 4: Verify by checking all interpretations
   All 8 interpretations match!

Step 5: Check Properties
  BDD1 satisfiable: True
  BDD1 valid: False
  BDD2 satisfiable: True
  BDD2 valid: False


## 10. Performance Analysis

### 10.1 Theoretical Complexity

**Space Complexity:**
- **Worst case:** O(2^n) - same as truth table
- **Best case:** O(n) - linear chains
- **Average case:** Often O(n^2) to O(n^3)

**Time Complexity:**

| Operation | Complexity |
|-----------|------------|
| Construction | O(2^n) worst, O(n) best |
| Reduction | O(n) with hash table |
| Apply (binary op) | O(\|f\| × \|g\|) worst |
| Evaluation | O(n) - traverse tree |
| Equivalence | O(1) after reduction |

### 10.2 Variable Ordering Impact

**Critical Factor:** Variable ordering dramatically affects BDD size!

**Example:** For formula (x₁ ∧ y₁) ∨ (x₂ ∧ y₂) ∨ ... ∨ (xₙ ∧ yₙ)

- **Good ordering:** [x₁, y₁, x₂, y₂, ..., xₙ, yₙ] → O(n) nodes
- **Bad ordering:** [x₁, x₂, ..., xₙ, y₁, y₂, ..., yₙ] → O(2^n) nodes

**Heuristics:**
- Put related variables together
- Dynamic variable ordering


In [47]:
# Run Performance Benchmarks
print("Performance Benchmarks")

import time

# Benchmark 1: Construction and Reduction
print("\n1. Construction & Reduction Speed\n")

test_cases = [
    ("p & q", "Simple conjunction"),
    ("p | (q & r)", "Textbook example"),
    ("(p & q) | (p & r)", "Complex formula"),
]

for formula_str, name in test_cases:
    start = time.time()
    bdd = create_bdd_from_string(formula_str)
    nodes_before = bdd.count_nodes()
    
    start_reduce = time.time()
    bdd.reduce()
    reduce_time = time.time() - start_reduce
    
    total_time = time.time() - start
    nodes_after = bdd.count_nodes()
    
    print(f"{name:20} | {nodes_before} → {nodes_after} nodes | "
          f"{total_time*1000:.2f}ms total, {reduce_time*1000:.2f}ms reduce")

# Benchmark 2: Operations
print("\n2. Operation Speed (average over 100 runs)\n")

bdd_p = create_bdd_from_string("p")
bdd_q = create_bdd_from_string("q")

ops = [
    ("AND", lambda: bdd_and(bdd_p, bdd_q)),
    ("OR", lambda: bdd_or(bdd_p, bdd_q)),
    ("XOR", lambda: bdd_xor(bdd_p, bdd_q)),
]

for op_name, op_func in ops:
    start = time.time()
    for _ in range(100):
        result = op_func()
    avg_time = (time.time() - start) / 100
    print(f"{op_name:10} | {avg_time*1000:.3f} ms")

# Benchmark 3: Scaling
print("\n3. Scaling with Variables\n")
print(f"{'Variables':<12} {'Truth Table':<15} {'BDD Nodes':<12} {'Ratio':<10}")

for n in range(2, 8):
    vars_list = [f"x{i}" for i in range(n)]
    formula_str = " | ".join([f"({vars_list[i]} & {vars_list[i+1]})" 
                              for i in range(0, n-1, 2) if i+1 < n])
    
    bdd = create_bdd_from_string(formula_str)
    bdd.reduce()
    nodes = bdd.count_nodes()
    truth_table_size = 2 ** n
    ratio = truth_table_size / nodes
    
    print(f"{n:<12} {truth_table_size:<15} {nodes:<12} {ratio:<10.1f}x")

print("\n Benchmarks complete!")

Performance Benchmarks

1. Construction & Reduction Speed

Simple conjunction   | 5 → 4 nodes | 0.15ms total, 0.04ms reduce
Textbook example     | 9 → 5 nodes | 0.13ms total, 0.04ms reduce
Complex formula      | 9 → 5 nodes | 0.12ms total, 0.03ms reduce

2. Operation Speed (average over 100 runs)

AND        | 0.022 ms
OR         | 0.016 ms
XOR        | 0.019 ms

3. Scaling with Variables

Variables    Truth Table     BDD Nodes    Ratio     
2            4               4            1.0       x
3            8               4            2.0       x
4            16              6            2.7       x
5            32              6            5.3       x
6            64              8            8.0       x
7            128             8            16.0      x

 Benchmarks complete!
